In [ ]:
!pip install ultralytics roboflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.2/922.2 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.1/83.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20

In [ ]:
from roboflow import Roboflow
from ultralytics import YOLO
import os


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
rf = Roboflow(api_key="API_KEY")  # Replace with your actual API key
project = rf.workspace("test-on9hk").project("vegetables-kacga")
version = project.version(5)  # Use the correct dataset version
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Vegetables-5 in yolov8:: 100%|██████████| 15910/15910 [00:02<00:00, 6839.16it/s] 


In [ ]:
dataset_path = dataset.location
print("Dataset downloaded to:", dataset_path)


Dataset downloaded to: /content/Vegetables-5


In [ ]:
import os
import shutil
import random


In [ ]:
dataset_path = dataset.location  # Path to dataset
images_dir = os.path.join(dataset_path, "train", "images")  # Image files
labels_dir = os.path.join(dataset_path, "train", "labels")  # Label files


In [ ]:
for split in ["train", "valid", "test"]:
    os.makedirs(os.path.join(dataset_path, split, "images"), exist_ok=True)
    os.makedirs(os.path.join(dataset_path, split, "labels"), exist_ok=True)


In [ ]:
image_files = [f for f in os.listdir(images_dir) if f.endswith(".jpg")]

random.shuffle(image_files)  # Shuffle data

train_split = int(0.8 * len(image_files))
val_split = int(0.9 * len(image_files))

train_files = image_files[:train_split]
val_files = image_files[train_split:val_split]
test_files = image_files[val_split:]

def move_files(file_list, split):
    for file in file_list:
        shutil.move(os.path.join(images_dir, file), os.path.join(dataset_path, split, "images", file))
        label_file = file.replace(".jpg", ".txt")  # Assuming labels are .txt
        if os.path.exists(os.path.join(labels_dir, label_file)):
            shutil.move(os.path.join(labels_dir, label_file), os.path.join(dataset_path, split, "labels", label_file))

move_files(train_files, "train")
move_files(val_files, "valid")
move_files(test_files, "test")

print("Dataset split successfully!")


Dataset split successfully!


In [ ]:
model = YOLO("yolov8n.pt")  # Using YOLOv8 Nano (lightweight model)

model.train(
    data=os.path.join(dataset_path, "data.yaml"),
    epochs=50,
    imgsz=640
)


100%|██████████| 6.25M/6.25M [00:00<00:00, 97.7MB/s]


New https://pypi.org/project/ultralytics/8.3.83 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.82 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/Vegetables-5/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fr

100%|██████████| 755k/755k [00:00<00:00, 19.0MB/s]


Overriding model.yaml nc=80 with nc=26

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

100%|██████████| 5.35M/5.35M [00:00<00:00, 82.9MB/s]


...

                   all        795       2670      0.834      0.752      0.835      0.562



50 epochs completed in 1.668 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.3MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.82 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,010,718 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:09<00:00,  2.65it/s]


                   all        795       2670      0.832      0.753      0.835      0.562
               avocado         41        144      0.865      0.931      0.946      0.593
                 beans         39         75      0.927      0.533      0.709      0.462
                  beet         34        122      0.797      0.902      0.914      0.701
           bell pepper         34         81      0.937      0.733      0.886      0.682
              broccoli         26         52      0.742      0.865      0.896      0.547
          brus capusta         23         57      0.704      0.543      0.676        0.4
               cabbage         25         79      0.778      0.532      0.694      0.373
                carrot         47        160      0.809      0.694      0.781      0.475
           cayliflower         23         49      0.915      0.878      0.923      0.636
                celery         16         41      0.749      0.732      0.811      0.493
                  cor

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x79e4d41f61d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.04104

In [ ]:
model = YOLO('runs/detect/train/weights/best.pt')

# Run the model on the uploaded image
results = model.predict(source='/content/Vegetables-5/test/images/--------------------------26_jpg.rf.273088c7bc13020c614cf055000feffd.jpg',save=True)


image 1/1 /content/Vegetables-5/test/images/--------------------------26_jpg.rf.273088c7bc13020c614cf055000feffd.jpg: 640x640 (no detections), 8.3ms
Speed: 3.2ms preprocess, 8.3ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict4


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Save best.pt to Google Drive
import shutil
shutil.copy('/content/runs/detect/train/weights/best.pt', '/content/drive/MyDrive/best.pt')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/MyDrive/best.pt'

In [ ]:
shutil.copy('/content/runs/detect/train/weights/best.pt', '/content/drive/MyDrive/best.pt')

'/content/drive/MyDrive/best.pt'

In [ ]:
shutil.copy('/content/runs/detect/train/weights/last.pt', '/content/drive/MyDrive/last.pt')

'/content/drive/MyDrive/last.pt'